# Results Log (Species Prediction Models)

## Regular Convolutional Neural Network

**Architecture:** (no regularization, filter width: 2, 80 filters)  
**[Conv/Pool/Drop] -> [Conv/Pool/Drop] -> [Conv/Pool/Drop]**  
Epochs: 60  
Training Accuracy: ~0.78 (training loss appears to still be declining)  
Validation Accuracy: ~0.54 (training loss/accuracy remained approximately the same since Epoch 40)  


## Convolutional Neural Network with Dilated Convolution Branch

**Architecture:**  
Regular Convolution Branch: **[Conv/Pool/Drop] -> [Conv/Pool/Drop] -> [Conv/Pool/Drop]**  
Dilated Convolution Branch: **[Conv-Dil: (1,2)/Pool/Drop] -> [Conv-Dil: (1,4)/Pool/Drop] -> [Conv-Dil: (1,8)/Pool/Drop]**  
Epochs: 25  
Training Accuracy: ~0.80  
Validation Accuracy: ~0.53  

**Architecture:**  
Regular Convolution Branch: **[Conv/Pool/Drop] -> [Conv/Pool/Drop] -> [Conv/Pool/Drop]**  
Dilated Convolution Branch: **[Conv-Dil: (1,8)/Pool/Drop] -> [Conv-Dil: (1,4)/Pool/Drop] -> [Conv-Dil: (1,2)/Pool/Drop]**  
Epochs: 40  
Training Accuracy: ~0.95 (@ 40 epochs), ~0.67 (@9 epochs)  
Validation Accuracy: ~0.50 (@40 epochs), ~0.58 (@9 epochs)  


**Architecture:** (no regularization!)  
Regular Convolution Branch: **[Conv/Pool/Drop] -> [Conv/Pool/Drop]**  
Dilated Convolution Branch: **[Conv-Dil: (1,4)/Pool/Drop] -> [Conv-Dil: (1,4)/Pool/Drop]**  
Stacked Output: **Stack/Reshape -> [Conv/Pool/Drop]**  
Epochs: 40  
Training Accuracy: ~0.94 (~0.73 @ 10 epochs)    
Validation Accuracy: ~0.49 (~0.54 @ 10 epochs)     

## LSTM Models
3 Current Models:  
1) LSTM only - involves breaking each promoter sequence into smaller subsequences (looking for short patterns, i.e. TF binding motifs)  
2) Conv + LSTM (sequential) - applies convolutions/pooling to "summarize" promoter sequence first and then applies LSTM to get longer range patterns across the entire sequence "summary"  
3) Conv + LSTM (parallel) - same as LSTM only model but with addition of separate branch consisting of multiple convolutional layers (representations of each branch are obtained via a dense layer for each branch and then concatenated)  

----

Architecture: (see below)   
Epochs: 20  
Training Accuracy: ~0.4-0.6 (highly variable...)  
Validation Accuracy: ~0.44 (validation loss doesn't appear to be declining)  

Training for more epochs...  
Epochs (total): 45  
Training Accuracy: ~0.75 (still highly variable...)  
Validation Accuracy: ~0.56  

----
Architecture: (see below, changed pooling dimensions to (1,5) and stride to (1,5))  
Epochs: 20  
Training Accuracy: ~0.5 (highly variable...)  
Validation Accuracy: ~0.4 (validation loss doesn't appear to be declining)  

In [ ]:
# LSTM block
conv1 = Conv2D(8,[4,5],activation='linear',
                name='convTrans_1',padding='valid')(dna)
leak1 = LeakyReLU(alpha=.001)(conv1)
pool1 = AveragePooling2D((1,2),strides=(1,2),name='AvgPoolTrans_1',padding='same')(leak1) 
conv2 = Conv2D(1,[1,1],activation='linear',name='convTrans1x1',padding='same')(pool1)
reshapedSeq = tf.reshape(tf.squeeze(conv2,[1,3]),[tf.shape(conv2)[0],31,8])
lstm1 = RNN(reshapedSeq,n_steps,n_hidden,100)

FC = Dense(50,activation='relu',name='representation')(lstm1)
preds = Dense(num_species,activation='softmax')(FC)

### LSTM (only)
Architecture: (see below)  
Epochs: ~20 (stopped)   
Patch Length: 10  
Training Accuracy: ~0.12-0.3 (highly variable...)    
Validation Accuracy: ~0.13    

Architecture: (see below, change 50s to 25)  
Epochs: 20  
Patch Length: 20  
Training Accuracy: ~0.2 (highly variable...)    
Validation Accuracy: ~0.14  

In [ ]:
reshapedDNA = tf.concat(tf.split(tf.squeeze(dna,[3]),50,2),0)
reshapedDNA = tf.reshape(reshapedDNA,[tf.shape(reshapedDNA)[0],tf.shape(reshapedDNA)[2],4])
lstm1 = RNN(reshapedDNA,n_steps,n_hidden,100)
reshapedLSTM = tf.reshape(lstm1,[tf.shape(dna)[0],50,100])

flat = Flatten()(reshapedLSTM)
FC = Dense(50,activation='relu',name='representation')(flat)
preds = Dense(num_species,activation='softmax')(FC)

### LSTM (2 Layers)

Architecture: (see below)  
Epochs: 20  
Training Accuracy: ///  
Validation Accuracy: ///  

In [ ]:
reshapedDNA = tf.concat(tf.split(tf.squeeze(dna,[3]),int(promoter_length/n_steps),2),0)
reshapedDNA = tf.reshape(reshapedDNA,[tf.shape(reshapedDNA)[0],tf.shape(reshapedDNA)[2],4])
with tf.variable_scope('LSTM1'):
    lstm1 = RNN(reshapedDNA,n_steps,n_hidden,32)

n_steps2 = int(promoter_length/n_steps)
reshapedLSTM1 = tf.reshape(lstm1,[tf.shape(dna)[0],n_steps2,32])
with tf.variable_scope('LSTM2'):
    lstm2 = RNN(reshapedLSTM1,n_steps2,n_hidden,64)

FC = Dense(50,activation='relu',name='representation')(lstm2)
preds = Dense(num_species,activation='softmax')(FC)

### LSTM + Conv (Parallel Branches)
Architecture: (see below)  
Epochs: ~20 (stopped)  
Training Accuracy: ~0.6-0.8 (highly variable...)  
Validation Accuracy: ~0.55  
